In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
os.chdir('gdrive/MyDrive/boddam-masters-project')

In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
"""
install
keras, numpy, tensorflow, scipy, Pillow

"""

from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import callbacks
from keras.utils import load_img, img_to_array
import pandas as pd
from PIL import Image
import glob
import numpy as np
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from numpy import load
import tensorflow as tf
#from tensorflow.keras.applications.inception_v3 import InceptionV3
#from tensorflow.keras.layers import Input

In [ ]:
train_data_dir = "data/train"
test_data_dir = "data/test"
data_width, data_height = 1000, 1000
#nb_train_samples = 8
#nb_validation_samples = 10
nb_filters1 = 2
nb_filters2 = 4
conv1_size = 5
conv2_size = 3
pool_size = 2
classes_num = 2
batch_size = 20
lr = 0.005
epochs = 5

tile_size = 256

In [ ]:
train_image_data_init = load('train_image_data.npz')
train_image_data = train_image_data_init['arr_0']
print(train_image_data.shape)

train_class_data_init = load('train_class_data.npz')
train_class_data = train_class_data_init['arr_0']
print(train_class_data.shape)


(331, 1000, 1000, 3)
(331, 2)


In [ ]:
train_image_data[0][0][0]

array([117,  60, 117], dtype=uint8)

In [ ]:
(trainX, valX, trainY, valY) = train_test_split(
    train_image_data, 
    train_class_data, 
    test_size=0.2, 
    random_state=42
)

In [ ]:
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow(
    x=trainX, 
    y=trainY,
    batch_size=batch_size 
    )
valid_generator = ImageDataGenerator().flow(valX, valY,
    batch_size=batch_size )


In [ ]:
test_image_data_init = load('test_image_data.npz')
test_image_data = test_image_data_init['arr_0']
print(test_image_data.shape)

test_class_data_init = load('test_class_data.npz')
test_class_data = test_class_data_init['arr_0']
print(test_class_data.shape)

(120, 1000, 1000, 3)
(120, 1)


In [ ]:
input_tensor = Input(shape=(data_width, data_height, 3))

model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)

model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])


96112376/96112376 [==============================] - 5s 0us/step


In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Physical devices cannot be modified after being initialized


In [ ]:
model.fit(
    train_generator, 
    validation_data=valid_generator,
    epochs=10
)

Epoch 1/10


ResourceExhaustedError: ignored

In [ ]:
tp = 0
tn = 0
fp = 0
fn = 0

result_array = model.predict(test_image_data)
for i in range(len(test_class_data)):
  expected = test_class_data[i]
  result = result_array[i]
  # index 0 is normal; index 1 is tumor
  if (expected[0] == 1):
    print("Label: Normal")
    if (result[0] == expected[0]):
      print("Predicted answer: Normal")
      tp += 1
    else:
      print("Predicted answer: Tumor")
      fn += 1
  else:
    print("Label: Tumor")
    if (result[0] == expected[0]):
      print("Predicted answer: Tumor")
      tn += 1
    else:
      print("Predicted answer: Normal")
      fp += 1

4/4 [==============================] - 2s 210ms/step
Label: Normal
Predicted answer: Normal
Label: Normal
Predicted answer: Normal
Label: Normal
Predicted answer: Tumor
Label: Normal
Predicted answer: Tumor
Label: Normal
Predicted answer: Tumor
Label: Normal
Predicted answer: Tumor
Label: Normal
Predicted answer: Tumor
Label: Normal
Predicted answer: Tumor
Label: Normal
Predicted answer: Tumor
Label: Normal
Predicted answer: Tumor
Label: Normal
Predicted answer: Tumor
Label: Normal
Predicted answer: Normal
Label: Normal
Predicted answer: Normal
Label: Normal
Predicted answer: Normal
Label: Normal
Predicted answer: Normal
Label: Normal
Predicted answer: Normal
Label: Normal
Predicted answer: Normal
Label: Normal
Predicted answer: Normal
Label: Normal
Predicted answer: Tumor
Label: Normal
Predicted answer: Tumor
Label: Normal
Predicted answer: Tumor
Label: Normal
Predicted answer: Tumor
Label: Normal
Predicted answer: Normal
Label: Normal
Predicted answer: Normal
Label: Normal
Predicted 

In [ ]:
"""
Check metrics
"""
print("True Positive: ", tp)
print("True Negative: ", tn)
print("False Positive: ", fp)
print("False Negative: ", fn)

if (tp + fp) != 0:
    precision = tp / (tp + fp)
    print("Precision: ", precision)
else:
    print("Precision cannot be calculated because (tp+fp)=0")
    print("F-measure also cannot be calculated")
if (tp + fn) != 0:
    recall = tp / (tp + fn)
    print("Recall: ", recall)
else:
    print("Recall cannot be calculated because (tp+fn)=0")
    print("F-measure also cannot be calculated")
if (tp + fp) != 0 and (tp + fn) != 0:
    f_measure = (2 * recall * precision) / (recall + precision)
    print("F-measure: ", f_measure)

True Positive:  66
True Negative:  0
False Positive:  0
False Negative:  54
Precision:  1.0
Recall:  0.55
F-measure:  0.7096774193548387


In [ ]:
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow(test_image_data, test_class_data, batch_size=1)
score = model.evaluate_generator(test_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


Test loss: 46.87295150756836
Test accuracy: 0.5833333134651184
